In [2]:
ID_GPU = 0  # set active GPU here (if no GPU is available, this has no effect - if specified GPU is unexisting, it forces CPU)

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(ID_GPU)

from keras import backend as K
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

Using TensorFlow backend.
/home/antoine/miniconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
import pandas as pd
from dateutil import parser
from pathlib import Path

import pickle
import numpy as np

from DP.battery import Battery
from DP.simulation import Simulation
from DP.DP_model import DP_model, learning_rate_reduction

from DP.constants import NUM_THREADS, NUM_MODELS

In [5]:
# list of site ids
id_list = [int(name.split('.')[0]) for name in os.listdir('../data/train')]
id_list = sorted(id_list)
print(id_list)

[1, 2, 3, 4, 7, 10, 11, 12, 29, 31, 32]


In [ ]:
for site_id in [29]:
    print("                     ")
    print("---------------------")
    print("site", site_id)
    print("---------------------")
    print("                     ")
    
    ## Fix random seed
    np.random.seed(1)
    tf.set_random_seed(2)

    simulation_dir = (Path(os.getcwd())/os.pardir).resolve()
    data_dir = simulation_dir/'data'

    # Load data from battery
    metadata_path = data_dir/'metadata.csv'
    metadata = pd.read_csv(metadata_path, index_col=0)
    battery_parameters = metadata.loc[site_id]

    batt = {}
    for batt_id in [1, 2]:
        batt[batt_id] = Battery(capacity=battery_parameters[f"Battery_{batt_id}_Capacity"] * 1000,
                                charging_power_limit=battery_parameters[f"Battery_{batt_id}_Power"] * 1000,
                                discharging_power_limit=-battery_parameters[f"Battery_{batt_id}_Power"] * 1000,
                                charging_efficiency=battery_parameters[f"Battery_{batt_id}_Charge_Efficiency"],
                                discharging_efficiency=battery_parameters[f"Battery_{batt_id}_Discharge_Efficiency"])

    # Load data from site
    df = {}
    for type_simu in ["train", "submit"]:
        site_data_path = data_dir/f"{type_simu}"/f"{site_id}.csv"

        df[type_simu] = pd.read_csv(site_data_path)
        df[type_simu].index = [parser.parse(date_str) for date_str in df[type_simu]['timestamp']]

    # factors for normalizing inputs
    norm = {}
    norm["battery"] = batt[2].capacity
    norm["load"] = np.max(df["train"].actual_consumption)
    norm["pv"] = np.max(df["train"].actual_pv)
    norm["price"] = np.max(df["train"].price_buy_00)

    # simu (used for precomputations)
    simu = {}
    for type_simu in ["train", "submit"]:
        simu[type_simu] = Simulation(df[type_simu], norm, num_threads=NUM_THREADS)
        simu[type_simu].precompute()

    # save norm in pickle file (to be used at deploy time)
    pickle_path = f'../output/DP/norms/norm_s{site_id}.p'
    with open(pickle_path, 'wb') as f:
        pickle.dump(norm, f)
        
    dp_model = {}
    for imodel in range(NUM_MODELS)[::-1]:
        print("                 ")
        print("   --------------")
        print("   imodel", imodel)
        print("   --------------")
        print("                 ")

        # create model
        if (imodel+1) in dp_model:
            dp_next = dp_model[imodel+1]
            dp_next.load(site_id)
        else:
            dp_next = None
        dp_model[imodel] = DP_model(imodel, simu, batt, norm, dp_next=dp_next)

        # train model
        dp_model[imodel].train()

        # save and clear
        dp_model[imodel].save(site_id)
        dp_model[imodel].clear()

                     
---------------------
site 29
---------------------
                     
                 
   --------------
   imodel 47
   --------------
                 
Epoch 1/1
3125/3125 [==============================] - 104s 33ms/step - loss: 0.7541 - Qvalue_MSE_loss: 0.7541 - Qvalue_MAP_loss: 0.1172 - score_fair_loss: -0.0647 - val_loss: 0.2921 - val_Qvalue_MSE_loss: 0.2921 - val_Qvalue_MAP_loss: 0.0885 - val_score_fair_loss: -0.0895
Epoch 2/2
3125/3125 [==============================] - 105s 34ms/step - loss: 0.2826 - Qvalue_MSE_loss: 0.2826 - Qvalue_MAP_loss: 0.0867 - score_fair_loss: -0.0849 - val_loss: 0.3272 - val_Qvalue_MSE_loss: 0.3272 - val_Qvalue_MAP_loss: 0.0860 - val_score_fair_loss: -0.1460
Epoch 3/3
3125/3125 [==============================] - 102s 33ms/step - loss: 0.2198 - Qvalue_MSE_loss: 0.2198 - Qvalue_MAP_loss: 0.0810 - score_fair_loss: -0.1149 - val_loss: 0.1449 - val_Qvalue_MSE_loss: 0.1449 - val_Qvalue_MAP_loss: 0.0747 - val_score_fair_loss: -0.10